In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("fivethirtyeight/the-ultimate-halloween-candy-power-ranking")

print("Path to dataset files:", path)

100%|██████████| 2.06k/2.06k [00:00<00:00, 2.52MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/fivethirtyeight/the-ultimate-halloween-candy-power-ranking/versions/1


In [5]:
import pandas as pd

df = pd.read_csv("candy-data.csv")
df.head()

,competitorname,chocolate,fruity,caramel,peanutyalmondy,nougat,crispedricewafer,hard,bar,pluribus,sugarpercent,pricepercent,winpercent
0,100 Grand,1,0,1,0,0,1,0,1,0,0.732,0.860,66.971725
1,3 Musketeers,1,0,0,0,1,0,0,1,0,0.604,0.511,67.602936
2,One dime,0,0,0,0,0,0,0,0,0,0.011,0.116,32.261086
3,One quarter,0,0,0,0,0,0,0,0,0,0.011,0.511,46.116505
4,Air Heads,0,1,0,0,0,0,0,0,0,0.906,0.511,52.341465


In [8]:
y = df['chocolate']

# Use all other relevant features (excluding chocolate and name)
features = [
    'fruity', 'caramel', 'peanutyalmondy', 'nougat',
    'crispedricewafer', 'sugarpercent', 'pricepercent', 'winpercent'
]

X = df[features]
print(X)

    fruity  caramel  peanutyalmondy  nougat  crispedricewafer  sugarpercent  \
0        0        1               0       0                 1         0.732   
1        0        0               0       1                 0         0.604   
2        0        0               0       0                 0         0.011   
3        0        0               0       0                 0         0.011   
4        1        0               0       0                 0         0.906   
..     ...      ...             ...     ...               ...           ...   
80       1        0               0       0                 0         0.220   
81       1        0               0       0                 0         0.093   
82       1        0               0       0                 0         0.313   
83       0        1               0       0                 0         0.186   
84       0        0               0       0                 1         0.872   

    pricepercent  winpercent  
0          0.860   6

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)


In [15]:
print(confusion_matrix(y_test, y_pred))
print('---------------------------------')
print(classification_report(y_test, y_pred))


[[8 1]
 [1 7]]
---------------------------------
              precision    recall  f1-score   support

           0       0.89      0.89      0.89         9
           1       0.88      0.88      0.88         8

    accuracy                           0.88        17
   macro avg       0.88      0.88      0.88        17
weighted avg       0.88      0.88      0.88        17



In [14]:
df_factor = pd.DataFrame(
    {
        'Features' : X.columns,
        'Importance' : model.coef_[0]
    }
).sort_values('Importance', ascending=False)

df_factor

,Features,Importance
6,pricepercent,0.649304
4,crispedricewafer,0.385176
7,winpercent,0.150495
2,peanutyalmondy,-0.002635
1,caramel,-0.107691
3,nougat,-0.183739
5,sugarpercent,-0.402859
0,fruity,-2.243465


from matplotlib import pyplot as plt
df_factor['Importance'].plot(kind='hist', bins=20, title='Importance')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
df_factor.groupby('Features').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df_factor['Importance'].plot(kind='line', figsize=(8, 4), title='Importance')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df_factor['Features'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df_factor, x='Importance', y='Features', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)